## Лабораторная работа 4: topic modeling

В данной лабораторной работе мы попытаемся обучить LDA-модель topic-моделингу на двух принципиально различных корпусах. 

В первой части вы познакомитесь с новыми возможностями библиотеки gensim, а также с возможностями парсинга в языке Python. Во второй части вам предстоит самостоятельно обучить LDA-модель и оценить качество её работы.

### Часть 1: topic modeling уровня /b/

Краеугольным камнем в машинном обучений в целом, и в NLP в частности, является выбор датасетов. Доселе мы использовали только стандартные, многократно обкатанные датасеты, но сегодня попробуем собрать свой. Практика работы с сырыми, необработанными данными весьма полезно! Заодно изучим возможности парсеров в Питоне.

Давайте напишем парсер, собирающий информацию о сообщения с русскомязычного анононимного форума (имиджборды) "Двач" ("Сосач", "Хиккач", если вам угодно). Двач, как и всякая имиджборда разделён на разделы (доски, борды), посвященные различным тематикам -- аниме, видеоигры, литература, религия... Каждая доска состоит из тем (тредов, топиков), которые создаются анонимными (при их желании) пользователями. Каждый тред посвящен обсуждению какого-то конкретного вопроса.

У некоторых разделов есть раздел архив, располагается он по адресу https://2ch.hk/(название раздела)/arch/, например для раздела музыка -- https://2ch.hk/mu/arch/. Если у вас есть минимальные навыки в языке html, а также если вы изучили документацию встроенного класса HTMLParser, то вам будет несложно написать два парсера.

Первый парсер (ArchiveParser) парсит HTML-страницу архива доски, вытягивает из неё ссылки на заархивированные треды, и скармливает их второму парсеру.

Второй парсер (ThreadParser) парсит HTML-страницу заархивированного треда, вытягивает из неё сообщения, складывает их вместе и собирает.

In [2]:
import time
import urllib.request
from html.parser import HTMLParser
from gensim.utils import simple_preprocess

def get_value_by_key(attrs, key):
    for (k, v) in attrs:
        if(k == key):
            return v;
    return None

class ArchiveParser(HTMLParser):
    flag = False
    threads = []
    limit = 200
    def handle_starttag(self, tag, attrs):
        if(self.limit > 0):
            if(tag == 'div'):
                cl = get_value_by_key(attrs, 'class')
                if (cl == 'box-data'):
                    self.flag = True;
            if(self.flag == True and tag == 'a'):
                href = get_value_by_key(attrs, 'href')
                if(len(href)>20):
                    print(href)
                    print(self.limit)
                    thread = parse_thread('https://2ch.hk' + href)
                    if(len(thread) > 10):
                        self.threads.append(thread)
                        self.limit = self.limit - 1
                    thread = []
        

    def handle_endtag(self, tag):
        if(tag == 'div'):
            self.flag = False;

    def handle_data(self, data):
        1+1
        
    def get_threads(self):
        return self.threads
    
    def clean(self):
        self.threads = []
        
parser = ArchiveParser()

def parse_archive(board = '/b/', page_number = 0):
    lines = []
    link = 'https://2ch.hk' + board + 'arch/' + str(page_number) +'.html'
    print(link)
    parser.limit = 100
    url = urllib.request.urlopen(link)
    for line in url.readlines():
        lines.append(line.decode('utf-8'))
    for line in lines:
        parser.feed(line)
    res = parser.get_threads()
    parser.clean()
    return res

In [3]:
class ThreadParser(HTMLParser):
    flag = False
    message = []
    messages = []
            
    def handle_starttag(self, tag, attrs):
        if(tag == 'blockquote'):
            self.flag = True;
            self.message = []
        

    def handle_endtag(self, tag):
        if(tag == 'blockquote'):
            self.flag = False
            if(self.message != []):
                self.messages.append(self.message)
            self.message = []

    def handle_data(self, data):
        if(self.flag):
            self.message.extend(simple_preprocess(data))
            
    def get_messages(self):
        return self.messages
    
    def clear_messages(self):
        flag = False
        self.message = []
        self.messages = []

t_parser = ThreadParser()

def parse_thread (link):
    url = urllib.request.urlopen(link)
    lines = []
    for line in url.readlines():
        lines.append(line.decode('utf-8', errors='ignore'))
    for line in lines:
        t_parser.feed(line)
    res = t_parser.get_messages()
    t_parser.clear_messages()
    #print(res)
    return res

Весьма много кода, верно? Если не потерялись, могли заметить функцию parse_archive, которая парсит страницу архива по доске и номеру страницы.


$\textbf{Задание.}$
Давайте применим её к каким-нибудь доскам. Выберите две доски двача, имеющие архив и скачайте архивы функцией parse_archive.

In [4]:
boards = ['/pr/', '/mus/'] #TODO: напишите название досок в формате /'доска'/, например /mu/ для Музыки
threads_by_topic = [parse_archive(board=board) for board in boards]

https://2ch.hk/pr/arch/0.html
/pr/arch/2016-05-04/res/708924.html
100
/pr/arch/2016-05-03/res/708740.html
100
/pr/arch/2016-05-11/res/708703.html
100
/pr/arch/2016-05-03/res/708541.html
99
/pr/arch/2016-05-02/res/708338.html
99
/pr/arch/2016-05-02/res/708024.html
99
/pr/arch/2016-05-02/res/708023.html
99
/pr/arch/2016-05-02/res/708010.html
99
/pr/arch/2016-05-09/res/707960.html
98
/pr/arch/2016-05-12/res/707882.html
97
/pr/arch/2016-05-03/res/707795.html
96
/pr/arch/2016-05-04/res/707611.html
95
/pr/arch/2016-05-11/res/707568.html
94
/pr/arch/2016-05-02/res/707426.html
93
/pr/arch/2016-06-08/res/707395.html
92
/pr/arch/2016-05-01/res/707282.html
91
/pr/arch/2016-05-11/res/707083.html
91
/pr/arch/2016-05-02/res/707079.html
90
/pr/arch/2016-05-01/res/707032.html
89
/pr/arch/2016-04-30/res/707031.html
89
/pr/arch/2016-05-01/res/706875.html
89
/pr/arch/2016-05-07/res/706843.html
88
/pr/arch/2016-05-02/res/706697.html
87
/pr/arch/2016-04-29/res/706407.html
87
/pr/arch/2016-07-26/res/706304.

Разделим наши данные на тренировочые и тестовые. Пусть каждый десятый тред попадает в тест-сет.

In [113]:
data = []
test = []

it = 0
for topic in threads_by_topic:
    for thread in topic:
        full = []
        for post in thread:
            full.extend(post)
        it = it + 1
        if(it % 10 == 0):
            test.append(full)
        else:
            data.append(full)

$\textbf{Задание.}$
В русском языке есть множество слов (частицы, междометия, всё что вы хотите), которые никак не отображают смысл слов и являются вспомогательными. Чтобы ваша модель работала лучше -- добавьте стоп-слова в список RUSSIAN_STOP_WORDS или в строку st_str. Эти слова отфильтруются из датасета перед тем, как модель начнет обучаться на датасете.

In [90]:
from gensim.utils import simple_preprocess
from gensim import corpora

RUSSIAN_STOP_WORDS = ['не', 'это', 'что','чем','это','как','https','нет','op','он','же','так','но','да','нет','или','и', 'на', "то", "бы", "все", "ты", "если", "по", "за", "там", "ну", "уже", "от", "есть","был", "даже", "было", "www", "com", "youtube", "из", "будет", "mp", "они", "только", "его", "она", "вот", 'просто', 'watch', 'кто', 'для', 'когда', 'тут', 'мне', 'где', 'мы', 'какой', 'может', 'меня', 'до', 'про', 'http', 'раз', 'почему', 'тебя', 'ещё', 'их', 'сейчас', 'тоже', 'во', 'чтобы', 'этого','без', 'него','вы','такой', 'можно', 'надо', 'нахуй', 'ли', 'потом', 'тред', 'больше', 'лучше', 'хуй', 'сам', 'после', 'со', 'лол', 'быть', 'нужно', 'этом', 'блять', 'бля', 'того', 'ничего', 'потому', 'нибудь', 'этот', 'под', 'через', 'ни', 'себе', 'ему', 'при', 'какие', 'пиздец', 'теперь', 'хоть', 'говно', 'тогда', 'блядь', 'кстати', 'че', 'себя', 'конечно', 'типа', 'много', 'том', 'нихуя', 'куда', 'всегда', 'нас', 'тот', 'ведь', 'эти', 'них', 'сука', 'пока', 'более', 'чего', 'html', 'были', 'всех', 'была', 'например', 'тем', 'ru', 'зачем', 'либо', 'вроде', 'всего', 'вопрос', 'php', 'против', 'здесь', 'ее', 'значит', 'совсем', 'сколько', 'им', 'org', 'именно', 'эту', 'делать', 'сделать', 'стоит', 'могу', 'хочу', 'спасибо', 'знаю', 'можешь']
st_str = "которых которые твой которой которого сих ком свой твоя этими слишком нами всему будь саму чаще ваше сами наш затем еще самих наши ту каждое мочь весь этим наша своих оба который зато те этих вся ваш такая теми ею которая нередко каждая также чему собой самими нем вами ими откуда такие тому та очень сама нему алло оно этому кому тобой таки твоё каждые твои мой нею самим ваши ваша кем мои однако сразу свое ними всё неё тех хотя всем тобою тебе одной другие этао само эта буду самой моё своей такое всею будут своего кого свои мог нам особенно её самому наше кроме вообще вон мною никто это"
RUSSIAN_STOP_WORDS.extend(st_str.split(' '))

data = [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in data]
id2word = corpora.Dictionary(data)

Создадим словарь и на его основе преобразуем слова в их id.

In [91]:
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

Обучим LDA-модель, используя библиотеку gensim. Зададим число тем равно числу скачанных досок.

In [92]:
from gensim.models import LdaModel

model = LdaModel(corpus, id2word=id2word, num_topics=len(threads_by_topic))

Теперь получим топ-10 самых используемых в каждой теме слов.

$\textbf{Задание.}$
Оцените насколько хорошо модель разделила темы.

In [93]:
for i in range(len(threads_by_topic)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

['один', 'играть', 'звук', 'время', 'писать', 'it', 'понял', 'лет', 'код', 'работает']
['играть', 'звук', 'один', 'анон', 'время', 'код', 'равно', 'звучит', 'понял', 'it']


На мой взгляд, получилось не очень. Все перемешалось.

Теперь прогоним тестовые треды на модели. Тестовый датасет разделен на n равных частей по 20 тредов, i-ая соответствует i-й доске.

In [94]:
other_corpus = [id2word.doc2bow(text) for text in [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in test]]

vector = [model[unseen_doc] for unseen_doc in other_corpus]
print(vector[0]) #вероятности принадлежности 0-го тестового треда в ту или иную тему

[(0, 0.83836347), (1, 0.16163658)]


In [95]:
i = 0

for res in vector:
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        print("Text #" + str(i) + ", topic #" + str(max_it) + str(", prob = " + str(res[max_it][1])))
    i = i + 1

Text #0, topic #0, prob = 0.83836347
Text #1, topic #0, prob = 0.8183342
Text #2, topic #0, prob = 0.8913399
Text #3, topic #0, prob = 0.74592173
Text #4, topic #0, prob = 0.8317572
Text #5, topic #0, prob = 0.7202715
Text #6, topic #0, prob = 0.59941894
Text #7, topic #0, prob = 0.62145555
Text #8, topic #0, prob = 0.8689005
Text #9, topic #0, prob = 0.8233881
Text #10, topic #0, prob = 0.80115616
Text #11, topic #0, prob = 0.59158224
Text #12, topic #1, prob = 0.7325945
Text #13, topic #0, prob = 0.5748442
Text #14, topic #1, prob = 0.76932573
Text #15, topic #1, prob = 0.5944841
Text #16, topic #0, prob = 0.80827755
Text #17, topic #0, prob = 0.687083
Text #18, topic #1, prob = 0.7352673
Text #19, topic #1, prob = 0.8026252


$\textbf{Задание.}$

Оцените результаты работы модели на тест сете. Если модель разделили данные плохо -- объясните, почему?

Не очень, много перепутал и отнес треды из музыкального треда в программирование. Возможно из-за большого количества одинакового лексикона и схожей направленности тредов – как научиться (играть на инструменте или как научиться програть на каком-либо языке). 

In [ ]:
Если прикинуть, что на этих ресурсах сидят в основном "вкатывальщики", то понятно почему. Видимо, и там и там похожие посты в стиле "хочу научиться", "хочу стать" и т.д. И в принципе из-за специфичности языка, на котором там общаются.

## Часть 2. А теперь нормальный датасет.

А теперь давайте воспользуемся более стандартным датасетом библиотеки sklreatn -- 20newsgroups, посвященную статьям на различные темы. Выберем 6 (7) -- Атеизм, яблочное железо, автомобили, хоккей, космос, христианство, ближний восток.

In [37]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism',
 'comp.sys.mac.hardware',
 'rec.autos',
 'rec.sport.hockey',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.mideast']
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories = categories)

$\textbf{Задание}$

Найдите библиотечный или опишите свой список ENGLISH_STOP_WORDS, убирающий не несущие никакого смысла английские слова.

In [38]:
from gensim.utils import simple_preprocess
from gensim import corpora

ENGLISH_STOP_WORDS = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

print('the' in ENGLISH_STOP_WORDS) 

data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece))) for piece in newsgroups_train.data]

True


$\textbf{Большое задание 1.}$

Для списка data создайте словарь id2word. Получите преобразованный TermDocumentFrequency список corpust и обучите на нем LDA модель.

In [39]:
from gensim.models import LdaModel, LsiModel

print(data[0])
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

model = LdaModel(corpus, id2word=id2word, num_topics=len(categories))

['considering', 'adding', 'floptical', 'drive', 'current', 'system', 'would', 'like', 'know', 'floptical', 'drives', 'recommended', 'quality', 'performance', 'preference', 'would', 'floptical', 'drives', 'capable', 'handling', 'floppies', 'handling', 'floppies', 'necessity', 'far', 'know', 'bit', 'iomega', 'floptical', 'infinity', 'floptical', 'drives', 'comments', 'recommendations', 'either', 'floptical', 'drives', 'worth', 'looking', 'purchased', 'mail', 'order', 'places', 'etc', 'thanks', 'advance', 'please', 'send', 'replies', 'directly', 'umsoroko', 'ccu', 'umanitoba', 'ca']
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 1), (15, 2), (16, 6), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2)]]


In [40]:
# Список тем:
for i in range(len(categories)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

['people', 'one', 'would', 'documentary', 'massacring', 'sane', 'reasonable', 'tony', 'invaders', 'many']
['smileys', 'stab', 'predicted', 'cares', 'one', 'anyway', 'could', 'psuvm', 'wrote', 'first']
['bible', 'people', 'would', 'one', 'tony', 'many', 'think', 'religious', 'reasonable', 'could']
['blues', 'one', 'conform', 'go', 'right', 'game', 'left', 'first', 'could', 'hawks']
['kurds', 'turkey', 'armenian', 'russian', 'caucasus', 'volunteers', 'bristol', 'power', 'source', 'moslems']
['think', 'gt', 'chart', 'manta', 'team', 'surprise', 'patrick', 'pretty', 'washington', 'american']
['go', 'would', 'lindros', 'ottawa', 'nothing', 'let', 'maybe', 'edu', 'rangers', 'beezer']


$\textbf{Большое задание 2.}$

В соответствии с тренировочными, обработайте тестовые данные.

Напишите функцию, которая с помощью модели возвращает наиболее вероятный id темы. С помощью F-меры оцените правильность работы модели.

In [41]:
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories = categories)

In [42]:
data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece))) for piece in newsgroups_test.data]
corpus = [id2word.doc2bow(text) for text in data]
vector = [model[unseen_doc] for unseen_doc in corpus]

In [43]:
def probability(text, vector):
    res = vector[text]

    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        return max_it

    return -1

In [44]:
from sklearn.metrics import f1_score
import itertools

replace = [0, 1, 2, 3, 4, 5, 6]

predicted = []

maximum = 0

replace_lists = list(itertools.permutations(replace))

for l in replace_lists:
    for i in range(len(newsgroups_test.data)):
        predicted.append(l[probability(i, vector)])

    score = f1_score(y_true=newsgroups_test.target, y_pred=predicted, average='micro')
    
    if score > maximum:
        maximum = score
        replace = l

    predicted = []

print(maximum)
print(replace)

0.19722534683164605
(5, 1, 4, 6, 2, 3, 0)


In [35]:
from sklearn.metrics import f1_score
import itertools

replace = [1, 4, 2, 5, 0, 3, 6]

predicted = []

for i in range(len(newsgroups_test.data)):
    predicted.append(replace[probability(i, vector)])

In [36]:
f1_score(y_true=newsgroups_test.target, y_pred=predicted, average='micro'

0.19460067491563554